# PySpark H.W Session 1



Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [1]:
import findspark
findspark.init()
import pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrame").getOrCreate()

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [20]:
df = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("walmart_stock.csv")

#### What are the column names?

In [23]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [24]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### Print out the first 5 columns.

In [25]:
df.take(5)

[Row(Date='2012-01-03', Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996),
 Row(Date='2012-01-04', Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475),
 Row(Date='2012-01-05', Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539),
 Row(Date='2012-01-06', Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922),
 Row(Date='2012-01-09', Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)]

#### Use describe() to learn about the DataFrame.

In [26]:
df.describe().show()

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|      null|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|2012-01-03|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|2016-12-30|         90.800003|        90.970001|            89.25|        90.4700

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [27]:
describe = df.describe()
describe.printSchema()

root
 |-- summary: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [28]:
from pyspark.sql.functions import round
describe.select(describe['summary'],
                round(describe['Open'], 2).alias('Open'),
               round(describe['High'], 2).alias('High'),
               round(describe['Low'], 2).alias('Low'),
               round(describe['Close'], 2).alias('Close'),
               round(describe['Volume'], 2).alias('Volume')).show()

+-------+------+------+------+------+----------+
|summary|  Open|  High|   Low| Close|    Volume|
+-------+------+------+------+------+----------+
|  count|1258.0|1258.0|1258.0|1258.0|    1258.0|
|   mean| 72.36| 72.84| 71.92| 72.39|8222093.48|
| stddev|  6.77|  6.77|  6.74|  6.76|4519780.84|
|    min| 56.39| 57.06|  56.3| 56.42| 2094900.0|
|    max|  90.8| 90.97| 89.25| 90.47| 8.08981E7|
+-------+------+------+------+------+----------+



#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [29]:
newDF = df.selectExpr("(High / Volume) * 100 as `HV Ratio`")
newDF.show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-4|
|6.290848613094555E-4|
|4.669412994783916...|
|7.367338463826307E-4|
|8.915604778943901E-4|
|8.644477436914569E-4|
|9.351828421515645E-4|
| 8.29141562102703E-4|
|7.712212102001476E-4|
|7.071764823529412E-4|
|0.001015495466386981|
|6.576354146362593E-4|
| 5.90145296180676E-4|
|8.547679455011844E-4|
|8.420709512685392E-4|
|0.001041448341728...|
| 8.31607541486243E-4|
|9.721183814992127E-4|
|8.029436027707578E-4|
|6.307432259386365E-4|
+--------------------+
only showing top 20 rows



#### What day had the Peak High in Price?

In [30]:
from pyspark.sql.functions import col, desc
df.orderBy(col("High").desc()).select(df['Date']).show(1)

+----------+
|      Date|
+----------+
|2015-01-13|
+----------+
only showing top 1 row



#### What is the mean of the Close column?

In [31]:
from pyspark.sql.functions import expr
df.select(expr("mean(Close) as `avg(Close)`")).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



#### What is the max and min of the Volume column?

In [32]:
from pyspark.sql.functions import min, max
df.select(min("Volume"), max("Volume")).show()

+-----------+-----------+
|min(Volume)|max(Volume)|
+-----------+-----------+
|    2094900|   80898100|
+-----------+-----------+



#### How many days was the Close lower than 60 dollars?

In [33]:
df.filter("Close < 60").count()

81

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [34]:
(df.filter("High > 80").count()) * 100/(df.count())

9.141494435612083

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [35]:
df.corr("High", "Volume")

-0.3384326061737161

#### What is the max High per year?

In [47]:
from pyspark.sql.functions import year
df2 = df.withColumn('Years', year(df['Date']))
df2.groupBy("Years").max()["years", "max(High)"].show()

+-----+---------+
|years|max(High)|
+-----+---------+
| 2015|90.970001|
| 2013|81.370003|
| 2014|88.089996|
| 2012|77.599998|
| 2016|75.190002|
+-----+---------+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [49]:
from pyspark.sql.functions import month
mon_df = df.withColumn('Month', month(df['Date'])).groupBy('Month').mean().orderBy('Month')
mon_df['Month', 'avg(Close)'].show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+

